In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from torch.optim import Adam
from torchvision.transforms import ToTensor
import statistics as stats
from pathlib import Path

from DIM.classification_stats import precision
from DIM.models import DeepInfoAsLatent, Encoder, Classifier

In [2]:
#!mkdir data data/tv models models/run1
cur_dir = '/notebooks/DockerShared/MINE'
data_dir = '/notebooks/DockerShared/MINE/data/tv'
root = Path(cur_dir)
model_path = root / Path('models/run1')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128
num_classes = 10
fully_supervised = False
reload = 0
epochs = 100
# image size 3, 32, 32; batch size must be an even number; shuffle must be True
ds = CIFAR10(data_dir, download=True, transform=ToTensor())
len_train = len(ds) // 10 * 9
len_test = len(ds) - len_train
train, test = random_split(ds, [len_train, len_test])
train_l = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
test_l = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True)

Files already downloaded and verified


In [3]:
if fully_supervised:
    classifier = nn.Sequential(Encoder(), Classifier()).to(device)
else:
    classifier = DeepInfoAsLatent('run1', '990').to(device)
    if reload > 0:
        classifier = torch.load(model_path / Path(f'w_dim{reload}.mdl'))
optim = Adam(classifier.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [4]:
for epoch in range(reload + 1, reload + epochs):
    ll = []
    batch = tqdm(train_l, total=len_train // batch_size)
    for x, target in batch:
        optim.zero_grad()
        x, target = x.to(device), target.to(device)
        y = classifier(x)
        loss = criterion(y, target)
        ll.append(loss.detach().item())
        batch.set_description(f'{epoch} Train Loss: {stats.mean(ll)}')
        loss.backward()
        optim.step()

    confusion = torch.zeros(num_classes, num_classes)
    batch = tqdm(test_l, total=len_test // batch_size)
    ll = []
    for x, target in batch:
        x, target = x.to(device), target.to(device)
        y = classifier(x)
        loss = criterion(y, target)
        ll.append(loss.detach().item())
        batch.set_description(f'{epoch} Test Loss: {stats.mean(ll)}')
        _, predicted = y.detach().max(1)
        for item in zip(predicted, target):
            confusion[item[0], item[1]] += 1

    precis = precision(confusion)
    print(precis)
    classifier_path = model_path / Path('w_dim' + str(epoch) + '.mdl')
    classifier_path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(classifier, str(classifier_path))

1 Test Loss: 2.2570445354168234: 100%|██████████| 39/39 [00:02<00:00, 15.79it/s]


(tensor([0.6699, 0.6224, 0.0408, 0.0000, 0.0082, 0.3645, 0.6968, 0.0020, 0.0000,
        0.0000]), 0.24419070512820512)


2 Test Loss: 2.225627581278483: 100%|██████████| 39/39 [00:02<00:00, 14.18it/s] 


(tensor([0.7033, 0.7245, 0.0204, 0.0000, 0.0348, 0.6248, 0.8028, 0.0138, 0.0000,
        0.0000]), 0.296474358974359)


3 Test Loss: 2.2046985565087733: 100%|██████████| 39/39 [00:02<00:00, 15.28it/s]


...

(tensor([0.6196, 0.6204, 0.3653, 0.3368, 0.5676, 0.6000, 0.6835, 0.6759, 0.7012,
        0.6275]), 0.5811298076923077)


98 Test Loss: 1.8924976740127954: 100%|██████████| 39/39 [00:02<00:00, 14.11it/s]


(tensor([0.6130, 0.6158, 0.3714, 0.3450, 0.5553, 0.6040, 0.6707, 0.6297, 0.6998,
        0.6518]), 0.5769230769230769)


99 Test Loss: 1.889027164532588: 100%|██████████| 39/39 [00:02<00:00, 15.99it/s] 


(tensor([0.6125, 0.6378, 0.3730, 0.3374, 0.5656, 0.6056, 0.6821, 0.6601, 0.6980,
        0.6242]), 0.5811298076923077)
